# <center>Machine Learning Project</center>

** **
## <center>*03.5 - Stack Emsemble*</center>

** **

The members of the `team` are:
- Ana Farinha - 20211514
- Francisco Capontes - 20211692
- Sofia Gomes - 20240848
- Rui Lourenço - 2021639



In [1]:
# Import libraries
import pandas as pd
import numpy as np

#make the split here
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import StratifiedKFold
import time

from utils import *
from utils_feature_selection import check_performace
from utils_dicts import *

import warnings
warnings.filterwarnings('ignore')

random_state = 68+1

## <span style="color:salmon"> 1. Import Dataset </span> 

In [2]:
# Import dataset
train_df = pd.read_csv('preprocessed_data/train_data.csv', index_col="Claim Identifier")

In [3]:
# Import dataset
test_df = pd.read_csv('./preprocessed_data/test_data.csv', index_col = 'Claim Identifier')

In [4]:
# Define Feature Selection: essential_features, reduced_features or []
feature_selection = essential_features

## <span style="color:salmon"> 2. Prepare Dataset </span> 

Define y as a target "Claim Injury Type Encoded" and X with all the other columns

In [5]:
X = train_df.drop(["Claim Injury Type Encoded"], axis = 1)
y = train_df["Claim Injury Type Encoded"]

## <span style="color:salmon"> 3. Stack Emsemble </span> 

In [6]:
cb_config ={
        "iterations": 300,
        "learning_rate": 0.7,
        "depth": 6,
        "l2_leaf_reg": 6,
        "bagging_temperature": 0.7,
        "random_state": random_state,
        "custom_metric":"F1",
        "early_stopping_rounds":50,
        "verbose":0
    }
xgb_config = {
        "n_estimators": 200,
        "learning_rate": 0.1,
        "max_depth": 7,
        "subsample": 0.9,
        "colsample_bytree": 0.9,
        "gamma": 0.3,
        "random_state": random_state,
        "objective":"multi:softmax",                  
        "num_class":8,                                
        "eval_metric":"merror",                                       
        "verbosity":0   
    }
dt_config = {
        "min_samples_split": 10,
        "min_samples_leaf": 4,
        "max_depth": 20,
        "criterion": "entropy",
        "random_state": random_state,
    }

In [7]:
base_models = [
    ("NaiveBayes", GaussianNB()),
    ("CatBoostClassifier", CatBoostClassifier(**cb_config)),
    ("XGBClassifier", XGBClassifier(**xgb_config)),
    ("DecisionTreeClassifier", DecisionTreeClassifier(**dt_config)),
]
meta_model = LogisticRegression(random_state=random_state)

In [8]:
model = StackingClassifier(estimators=base_models, final_estimator=meta_model)

In [9]:
check_performace(model,X,y,numerical_features,essential_features,n_folds = 3, random_state=random_state)

Fold 1 train F1 score: 0.3127
Fold 1 validation F1 score: 0.3019
------------------------------
Fold 2 train F1 score: 0.3115
Fold 2 validation F1 score: 0.3075
------------------------------
Fold 3 train F1 score: 0.3062
Fold 3 validation F1 score: 0.3003
------------------------------
Average Train F1 score: 0.31014507167191874
Average Validation F1 score: 0.30320346213553373


In [10]:
check_performace(model,X,y,numerical_features,[],n_folds = 3, random_state=random_state)

Fold 1 train F1 score: 0.3480
Fold 1 validation F1 score: 0.2661
------------------------------
Fold 2 train F1 score: 0.3290
Fold 2 validation F1 score: 0.2741
------------------------------
Fold 3 train F1 score: 0.3654
Fold 3 validation F1 score: 0.2752
------------------------------
Average Train F1 score: 0.34745366681530165
Average Validation F1 score: 0.27178792135197133


#### <span style="color:salmon"> 3.1  Evaluate the model </span> 


In [11]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.2, stratify = y, shuffle = True, random_state=random_state)
X_train_to_preprocess=X_train.copy()

In [12]:
remove_outliers(X_train)
X_train, X_val = apply_frequency_encoding(X_train, X_val)
NA_imputer(X_train,X_val)
create_new_features(X_train,X_val)

In [13]:
scaler = StandardScaler().fit(X_train[numerical_features])
X_train[numerical_features]  = scaler.transform(X_train[numerical_features])
X_val[numerical_features]  = scaler.transform(X_val[numerical_features]) 

In [16]:
drop_list = ["Average Weekly Wage"]
if feature_selection != []:
    for col in X_train.columns:
        if col not in feature_selection:
            drop_list.append(col)
X_train = X_train.drop(drop_list, axis=1)
X_val = X_val.drop(drop_list, axis=1)

In [17]:
model.fit(X_train, y_train)

StackingClassifier(estimators=[('NaiveBayes', GaussianNB()),
                               ('CatBoostClassifier',
                                <catboost.core.CatBoostClassifier object at 0x000002991B4E6FD0>),
                               ('XGBClassifier',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.9, device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              ev...
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=200, n_jobs=None,
                                              num_class=8,
                                              num_parallel_tree=None, ...)),
                               ('DecisionTreeClassifier',
                                DecisionTreeClassifier(criterion='entropy',
                                                       max_depth=20,
                                                       min_samples_leaf=4,
                                                       min_samples_split=10,
                                                       random_state=69))],
                   final_estimator=LogisticRegression(random_state=69))

In [18]:
y_train_pred = model.predict(X_train)

In [19]:
y_val_pred = model.predict(X_val)

In [20]:
class_mapping = {
    0:'1. CANCELLED', 
    1:'2. NON-COMP',
    2:'3. MED ONLY', 
    3:'4. TEMPORARY',
    4:'5. PPD SCH LOSS', 
    5:'6. PPD NSL', 
    6:'7. PTD', 
    7:'8. DEATH'
}

# Use the values from class_mapping as the target names
target_names = list(class_mapping.values())

Compute confusion matrix to evaluate the accuracy of a classification

In [21]:
# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_train, y_train_pred))
print("\nClassification Report:")
print(classification_report(y_train, y_train_pred, target_names=target_names))

Confusion Matrix:
[[  2780   6314    148    720     18      0      0      0]
 [  1458 220381   1411   9361    251      0      0      0]
 [    47  31804   4048  17095   2130      1      0      0]
 [    55  27498   2335  82719   6198      0      0      0]
 [     0   1568    601  18465  17990      0      0      0]
 [     0      5     18   2829    502     15      0      0]
 [     0      0      1     74      3      0      0      0]
 [     0     70     23    278      5      0      0      0]]

Classification Report:
                 precision    recall  f1-score   support

   1. CANCELLED       0.64      0.28      0.39      9980
    2. NON-COMP       0.77      0.95      0.85    232862
    3. MED ONLY       0.47      0.07      0.13     55125
   4. TEMPORARY       0.63      0.70      0.66    118805
5. PPD SCH LOSS       0.66      0.47      0.55     38624
     6. PPD NSL       0.94      0.00      0.01      3369
         7. PTD       0.00      0.00      0.00        78
       8. DEATH       0.00  

In [22]:
# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred, target_names=target_names))

Confusion Matrix:
[[  673  1592    43   183     4     0     0     0]
 [  390 54937   404  2389    96     0     0     0]
 [    9  8009   856  4403   504     0     0     0]
 [   13  6991   677 20391  1628     2     0     0]
 [    1   386   165  4684  4420     0     0     0]
 [    0     1     5   718   118     0     0     0]
 [    0     0     0    17     2     0     0     0]
 [    0    11     5    77     1     0     0     0]]

Classification Report:
                 precision    recall  f1-score   support

   1. CANCELLED       0.62      0.27      0.38      2495
    2. NON-COMP       0.76      0.94      0.84     58216
    3. MED ONLY       0.40      0.06      0.11     13781
   4. TEMPORARY       0.62      0.69      0.65     29702
5. PPD SCH LOSS       0.65      0.46      0.54      9656
     6. PPD NSL       0.00      0.00      0.00       842
         7. PTD       0.00      0.00      0.00        19
       8. DEATH       0.00      0.00      0.00        94

       accuracy                   

## <span style="color:salmon"> 4. Test Predictions </span> 

Make validation predictions:

In [23]:
remove_outliers(X_train_to_preprocess)
X_train_to_preprocess, test_df = apply_frequency_encoding(X_train_to_preprocess, test_df)
NA_imputer(X_train_to_preprocess, test_df)
create_new_features(X_train_to_preprocess, test_df)

In [24]:
test_df[numerical_features]  = scaler.transform(test_df[numerical_features])  

In [25]:
test_df = test_df.drop(drop_list, axis=1)

In [26]:
# Make validation predictions
y_test_pred = model.predict(test_df)
y_test_pred = y_test_pred.ravel()

In [27]:
y_test_final = np.array([class_mapping[i] for i in y_test_pred])

In [28]:
test_id = test_df.index

In [29]:
submission_df = pd.DataFrame({
    'Claim Identifier': test_id,
    'Claim Injury Type': y_test_final
})

In [30]:
if False:
    version = version_control()
    submission_df.to_csv(f'./submissions/Group49_Version{version:02}.csv', index=False)